# All in one (attempting to add everything in one place)
**keeping the following cells**

STEP 1: Descriptor Computation & Save

STEP 2: LightGBM Training (v6)

STEP 2.2: Feature Importance-Based Masking

STEP 3: SHAP Prediction & Explanation (with top features)

STEP 4: Utility Functions (summarize_prediction, etc.)

STEP 5: Example Usage

## STEP 1: Data Loading & Descriptor Computation

In [1]:
import pandas as pd
import numpy as np
import os
from mordred import Calculator, descriptors
from rdkit import Chem
from tqdm import tqdm

tqdm.pandas()

# Define paths
DATA_PATH = "tox21_lightgb_pipeline/Data_v6/processed/tox21.csv"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
os.makedirs(SAVE_DIR, exist_ok=True)

# Load data
df = pd.read_csv(DATA_PATH)
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# Filter valid SMILES
df = df[df["smiles"].apply(lambda x: isinstance(x, str))]

# Compute descriptors
calc = Calculator(descriptors, ignore_3D=True)

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol)
    except:
        return None

mols = df["smiles"].progress_apply(Chem.MolFromSmiles)
descs = mols.progress_apply(lambda m: calc(m) if m is not None else None)
desc_df = pd.DataFrame([d.asdict() if d is not None else {} for d in descs])
desc_df = desc_df.dropna(axis=1)

# Save feature names
with open(os.path.join(SAVE_DIR, "feature_names.txt"), "w") as f:
    f.write("\n".join(desc_df.columns))

# Extract X and 
X = desc_df.values.astype(np.float32)
Y = df[label_cols].values.astype(np.float32)
label_mask = ~df[label_cols].isna().values

np.save(os.path.join(SAVE_DIR, "X.npy"), X)
np.save(os.path.join(SAVE_DIR, "Y.npy"), Y)
np.save(os.path.join(SAVE_DIR, "label_mask.npy"), label_mask)

  1%|          | 70/7831 [00:02<04:45, 27.18it/s][22:26:09] WARNING: not removing hydrogen atom without neighbors
[22:26:09] WARNING: not removing hydrogen atom without neighbors
[22:26:09] WARNING: not removing hydrogen atom without neighbors
  1%|▏         | 114/7831 [00:04<09:16, 13.87it/s]d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\lord\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 7831/7831 [06:27<00:00, 20.22it/s]


### Handle NaN values in Y labels (classification targets)

In [11]:
import numpy as np
import os

# Reload just to be sure
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))

# Strategy: Replace NaN in Y with -1 (we won't train on those)
# Alternatively, you can use `label_mask` directly for filtering later
Y_cleaned = np.where(label_mask, Y, -1)

# Save cleaned Y for model training
np.save(os.path.join(SAVE_DIR, "Y.npy"), Y_cleaned)

print("✅ Y_cleaned.npy saved. NaNs replaced with -1 for masking during model training.")


✅ Y_cleaned.npy saved. NaNs replaced with -1 for masking during model training.


## STEP 2: Model Training (LightGBM)

In [ ]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
import joblib
import optuna
import numpy as np
import pandas as pd
import os

# === Load precomputed data ===
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
os.makedirs(MODEL_PATH, exist_ok=True)

X = np.load(os.path.join(SAVE_DIR, "X.npy"))
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    feature_names = f.read().splitlines()

# === Define labels ===
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# === Training Setup ===
results = []
feature_mask_dict = {}
threshold_dict = {}

for i, label_name in enumerate(label_cols):
    print(f"\n📍 Training for label: {label_name}")

    # Subset valid rows for this label
    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    # Split the data
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, random_state=42, stratify=y_valid
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )

    # === Feature Selection ===
    selector_model = lgb.LGBMClassifier(n_estimators=200, random_state=42)
    selector_model.fit(X_train, y_train)
    selector = SelectFromModel(selector_model, threshold="median", prefit=True)
    kept_idx = selector.get_support(indices=True)

    X_train_fs = selector.transform(X_train)
    X_val_fs = selector.transform(X_val)
    X_test_fs = selector.transform(X_test)

    feature_mask_dict[label_name] = kept_idx

    # === Class Imbalance Handling ===
    n_pos = np.sum(y_train == 1)
    n_neg = np.sum(y_train == 0)
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1.0

    # === Hyperparameter Tuning with Optuna ===
    dtrain = lgb.Dataset(X_train_fs, label=y_train)
    dval = lgb.Dataset(X_val_fs, label=y_val)

    def objective(trial):
        params = {
            "objective": "binary",
            "metric": "auc",
            "boosting_type": "gbdt",
            "verbosity": -1,
            "feature_pre_filter": False,
            "scale_pos_weight": scale_pos_weight,
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "num_leaves": trial.suggest_int("num_leaves", 20, 80),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
            "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
        }
        model = lgb.train(
            params, dtrain, valid_sets=[dval], callbacks=[
                early_stopping(stopping_rounds=30),
                log_evaluation(period=0)
            ]
        )
        preds = model.predict(X_val_fs)
        return roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    best_params = study.best_params
    best_params.update({
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "scale_pos_weight": scale_pos_weight
    })

    # === Final Model Training ===
    final_model = lgb.LGBMClassifier(**best_params, n_estimators=500)
    final_model.fit(X_train_fs, y_train)

    # === Threshold Selection ===
    preds_val = final_model.predict_proba(X_val_fs)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_val, preds_val)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    best_thresh = thresholds[best_idx]

    # === Final Evaluation ===
    y_pred_bin = (final_model.predict_proba(X_test_fs)[:, 1] >= best_thresh).astype(int)
    auc = roc_auc_score(y_test, final_model.predict_proba(X_test_fs)[:, 1])
    acc = accuracy_score(y_test, y_pred_bin)

    # === Save Artifacts ===
    joblib.dump(final_model, os.path.join(MODEL_PATH, f"{label_name}.pkl"))
    threshold_dict[label_name] = best_thresh

    results.append({
        "label": label_name,
        "auc": auc,
        "accuracy": acc,
        "n_train": len(y_train),
        "n_val": len(y_val),
        "n_test": len(y_test),
        "best_threshold": best_thresh,
        "num_features": len(kept_idx),
        "scale_pos_weight": scale_pos_weight
    })

# === Save All Utilities ===
joblib.dump(threshold_dict, os.path.join(MODEL_PATH, "thresholds.pkl"))
joblib.dump(feature_mask_dict, os.path.join(MODEL_PATH, "feature_masks.pkl"))
pd.DataFrame(results).to_csv(os.path.join(MODEL_PATH, "training_metrics.csv"), index=False)


📍 Training for label: NR-AR


[I 2025-08-04 22:45:42,281] A new study created in memory with name: no-name-48feed37-b7c1-4dc8-8321-f0259a752ea9
[I 2025-08-04 22:45:42,690] Trial 0 finished with value: 0.6950104432582965 and parameters: {'learning_rate': 0.12548983540874797, 'num_leaves': 61, 'max_depth': 3, 'min_child_samples': 27, 'feature_fraction': 0.81759911359066, 'bagging_fraction': 0.8221146941074708, 'lambda_l1': 0.0699981664681033, 'lambda_l2': 0.36492819595499515}. Best is trial 0 with value: 0.6950104432582965.


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[82]	valid_0's auc: 0.69501
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid_0's auc: 0.727942


[I 2025-08-04 22:45:42,947] Trial 1 finished with value: 0.7279415177535391 and parameters: {'learning_rate': 0.08230155003401189, 'num_leaves': 31, 'max_depth': 7, 'min_child_samples': 29, 'feature_fraction': 0.6131242838472909, 'bagging_fraction': 0.8964264489053455, 'lambda_l1': 0.28480761237641183, 'lambda_l2': 0.3633740275928765}. Best is trial 1 with value: 0.7279415177535391.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's auc: 0.723091


...

Early stopping, best iteration is:
[62]	valid_0's auc: 0.885714
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 22:49:34,716] Trial 28 finished with value: 0.8673790776152981 and parameters: {'learning_rate': 0.14210232297837835, 'num_leaves': 40, 'max_depth': 7, 'min_child_samples': 44, 'feature_fraction': 0.8006665592646889, 'bagging_fraction': 0.8337601916691072, 'lambda_l1': 0.8985050093666247, 'lambda_l2': 0.7967267440386506}. Best is trial 24 with value: 0.9016497937757781.


Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.867379
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 22:49:35,034] Trial 29 finished with value: 0.9050618672665917 and parameters: {'learning_rate': 0.1201703155829826, 'num_leaves': 34, 'max_depth': 5, 'min_child_samples': 33, 'feature_fraction': 0.9242927584196634, 'bagging_fraction': 0.6896252731160479, 'lambda_l1': 0.7960188286042771, 'lambda_l2': 0.8896506764156475}. Best is trial 29 with value: 0.9050618672665917.


Early stopping, best iteration is:
[40]	valid_0's auc: 0.905062


### 2.1 eval 

In [12]:
import pandas as pd
import os

# Create results DataFrame
results_df = pd.DataFrame(results)

# Sort by AUC (optional)
results_df = results_df.sort_values(by="auc", ascending=False)

# Display results
print("📈 Summary of LightGBM model performance:")
display(results_df)

# Save to CSV
results_df.to_csv(os.path.join(MODEL_PATH, "training_metrics.csv"), index=False)


📈 Summary of LightGBM model performance:


,label,auc,accuracy,n_train,n_val,n_test,best_threshold,num_features,scale_pos_weight
10,SR-MMP,0.948920,0.924269,4648,581,581,0.344759,812,5.332425
8,SR-ATAD5,0.886605,0.951977,5657,707,708,0.140595,931,25.810427
2,NR-AhR,0.881095,0.893130,5239,655,655,0.487913,807,7.532573
6,NR-PPAR-gamma,0.877636,0.964341,5160,645,645,0.653218,907,33.630872
11,SR-p53,0.858378,0.920354,5419,677,678,0.134669,828,15.032544
9,SR-HSE,0.853877,0.927357,5173,647,647,0.081956,819,16.359060
7,SR-ARE,0.832675,0.835616,4665,583,584,0.211113,831,5.187003
5,NR-ER-LBD,0.822563,0.965517,5564,695,696,0.657139,964,18.871429
3,NR-Aromatase,0.822122,0.953688,4656,582,583,0.373381,836,18.400000
0,NR-AR,0.813450,0.971114,5812,726,727,0.978191,943,22.530364


## 2*: Train LightGBM Models Using All Features (No Feature Selection)

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
import optuna
import joblib

X = np.load("tox21_lightgb_pipeline/Data_v6/processed/X.npy")
Y = np.load("tox21_lightgb_pipeline/Data_v6/processed/Y.npy")
label_mask = np.load("tox21_lightgb_pipeline/Data_v6/processed/label_mask.npy")

with open("tox21_lightgb_pipeline/Data_v6/processed/feature_names.txt") as f:
    feature_names = f.read().splitlines()

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
os.makedirs(MODEL_PATH, exist_ok=True)

threshold_dict = {}

for i, label_name in enumerate(label_cols):
    print(f"📍 Training for label: {label_name}")

    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, stratify=y_valid, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )

    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)

    def objective(trial):
        params = {
            "objective": "binary",
            "metric": "auc",
            "boosting_type": "gbdt",
            "verbosity": -1,
            "feature_pre_filter": False,
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "num_leaves": trial.suggest_int("num_leaves", 20, 80),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
            "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
            "scale_pos_weight": sum(y_train == 0) / sum(y_train == 1),
        }
        model = lgb.train(
            params,
            dtrain,
            valid_sets=[dval],
            callbacks=[lgb.early_stopping(stopping_rounds=30), lgb.log_evaluation(period=0)]
        )
        preds = model.predict(X_val)
        return roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    best_params = study.best_params
    best_params.update({
        "objective": "binary", "metric": "auc", "verbosity": -1, "boosting_type": "gbdt"
    })

    final_model = lgb.LGBMClassifier(**best_params, n_estimators=500)
    final_model.fit(X_train, y_train)

    # Save model
    joblib.dump(final_model, os.path.join(MODEL_PATH, f"{label_name}.pkl"))

    # Compute and save threshold
    preds_val = final_model.predict_proba(X_val)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_val, preds_val)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
    best_thresh = thresholds[np.argmax(f1_scores)]
    threshold_dict[label_name] = best_thresh

# Save thresholds
joblib.dump(threshold_dict, os.path.join(MODEL_PATH, "thresholds.pkl"))


[I 2025-08-04 23:00:49,680] A new study created in memory with name: no-name-e51e3d81-02cb-4a3a-b48a-d9759f256712


📍 Training for label: NR-AR
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 23:00:50,599] Trial 0 finished with value: 0.7320491993501973 and parameters: {'learning_rate': 0.13402338975175815, 'num_leaves': 47, 'max_depth': 9, 'min_child_samples': 42, 'feature_fraction': 0.9096214406960866, 'bagging_fraction': 0.7371821929763615, 'lambda_l1': 0.9723280700534054, 'lambda_l2': 0.21795035154360598}. Best is trial 0 with value: 0.7320491993501973.


Early stopping, best iteration is:
[18]	valid_0's auc: 0.732049
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 23:00:50,879] Trial 1 finished with value: 0.7464376885588304 and parameters: {'learning_rate': 0.19423906055200094, 'num_leaves': 50, 'max_depth': 4, 'min_child_samples': 13, 'feature_fraction': 0.8470602632442801, 'bagging_fraction': 0.7447417106829443, 'lambda_l1': 0.52595586613831, 'lambda_l2': 0.25784463525149903}. Best is trial 1 with value: 0.7464376885588304.


...

[I 2025-08-04 23:05:51,982] Trial 28 finished with value: 0.8716535433070867 and parameters: {'learning_rate': 0.10914922801102256, 'num_leaves': 76, 'max_depth': 11, 'min_child_samples': 27, 'feature_fraction': 0.7779255804671497, 'bagging_fraction': 0.9915533909843324, 'lambda_l1': 0.6978004546415254, 'lambda_l2': 0.25326232752216093}. Best is trial 17 with value: 0.8999250093738282.


Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.871654
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 23:05:52,590] Trial 29 finished with value: 0.8873265841769779 and parameters: {'learning_rate': 0.18647945193848248, 'num_leaves': 80, 'max_depth': 8, 'min_child_samples': 37, 'feature_fraction': 0.6190211184784666, 'bagging_fraction': 0.9974407664668596, 'lambda_l1': 0.44248548131837645, 'lambda_l2': 0.436063766727822}. Best is trial 17 with value: 0.8999250093738282.


Early stopping, best iteration is:
[25]	valid_0's auc: 0.887327


['tox21_lightgb_pipeline/models/v6\\thresholds.pkl']

In [3]:
import numpy as np
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# Load saved data
X = np.load("tox21_lightgb_pipeline/Data_v6/processed/X.npy")
Y = np.load("tox21_lightgb_pipeline/Data_v6/processed/Y.npy")
label_mask = np.load("tox21_lightgb_pipeline/Data_v6/processed/label_mask.npy")

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

eval_results = []

for i, label_name in enumerate(label_cols):
    print(f"📊 Evaluating: {label_name}")
    
    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    # Same train/val/test split as training
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, stratify=y_valid, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )

    model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
    threshold = thresholds[label_name]

    prob_test = model.predict_proba(X_test)[:, 1]
    pred_test = (prob_test >= threshold).astype(int)

    auc = roc_auc_score(y_test, prob_test)
    acc = accuracy_score(y_test, pred_test)
    f1 = f1_score(y_test, pred_test)

    eval_results.append({
        "label": label_name,
        "test_auc": round(auc, 4),
        "test_accuracy": round(acc, 4),
        "test_f1": round(f1, 4),
        "n_test": len(y_test)
    })

# Save evaluation results
eval_df = pd.DataFrame(eval_results)
eval_df = eval_df.sort_values(by="test_auc", ascending=False)

print("\n✅ Evaluation Summary:")
display(eval_df)

eval_df.to_csv(os.path.join(MODEL_PATH, "evaluation_metrics.csv"), index=False)


📊 Evaluating: NR-AR
📊 Evaluating: NR-AR-LBD
📊 Evaluating: NR-AhR
📊 Evaluating: NR-Aromatase
📊 Evaluating: NR-ER
📊 Evaluating: NR-ER-LBD
📊 Evaluating: NR-PPAR-gamma
📊 Evaluating: SR-ARE
📊 Evaluating: SR-ATAD5
📊 Evaluating: SR-HSE
📊 Evaluating: SR-MMP
📊 Evaluating: SR-p53

✅ Evaluation Summary:


,label,test_auc,test_accuracy,test_f1,n_test
10,SR-MMP,0.9484,0.9329,0.7821,581
6,NR-PPAR-gamma,0.9339,0.9736,0.3200,645
8,SR-ATAD5,0.9142,0.9576,0.5000,708
2,NR-AhR,0.8938,0.9069,0.5734,655
11,SR-p53,0.8587,0.9041,0.3564,678
9,SR-HSE,0.8505,0.9413,0.4242,647
5,NR-ER-LBD,0.8483,0.9569,0.5312,696
7,SR-ARE,0.8343,0.8493,0.5368,584
3,NR-Aromatase,0.8318,0.9400,0.4068,583
0,NR-AR,0.8266,0.9684,0.5490,727


## 2**: Feature Selection Lgbm

In [1]:
import os
import joblib
import numpy as np
import pandas as pd

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"

# Load data and metadata
X = np.load(os.path.join(SAVE_DIR, "X.npy"))
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    feature_names = f.read().splitlines()

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# Dictionary to hold best features
feature_mask_dict = {}
importance_df = []

# Loop through each label
for i, label_name in enumerate(label_cols):
    print(f"\n🔍 Selecting features for: {label_name}")

    # Load model
    model_path = os.path.join(MODEL_PATH, f"{label_name}.pkl")
    model = joblib.load(model_path)

    # Get training subset used for this label
    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    # Get feature importances (use gain or split)
    importances = model.feature_importances_
    top_k = min(300, int(len(importances) * 0.2))  # top 20% or 300 max
    top_idx = np.argsort(importances)[::-1][:top_k]

    feature_mask_dict[label_name] = top_idx

    importance_df.append(pd.DataFrame({
        "label": label_name,
        "feature": np.array(feature_names)[top_idx],
        "importance": importances[top_idx]
    }))

# Save
joblib.dump(feature_mask_dict, os.path.join(MODEL_PATH, "feature_masks.pkl"))
importance_df = pd.concat(importance_df)
importance_df.to_csv(os.path.join(MODEL_PATH, "top_features_per_label.csv"), index=False)

print("\n✅ Feature masks and importances saved.")


🔍 Selecting features for: NR-AR

🔍 Selecting features for: NR-AR-LBD

🔍 Selecting features for: NR-AhR

🔍 Selecting features for: NR-Aromatase

🔍 Selecting features for: NR-ER

🔍 Selecting features for: NR-ER-LBD

🔍 Selecting features for: NR-PPAR-gamma

🔍 Selecting features for: SR-ARE

🔍 Selecting features for: SR-ATAD5

🔍 Selecting features for: SR-HSE

🔍 Selecting features for: SR-MMP

🔍 Selecting features for: SR-p53

✅ Feature masks and importances saved.


## step 3: Predict + SHAP Explanation

In [15]:
import shap
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors as rdmd

from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

from rdkit.Chem import MolStandardize


MODEL_PATH = 'tox21_lightgb_pipeline/models/v_base'

import warnings
warnings.filterwarnings("ignore")

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol).asdict()
    except:
        return None

feature_masks = joblib.load(os.path.join(MODEL_PATH, "feature_masks.pkl"))
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()


def predict_and_explain_all_labels(smiles):
    desc = compute_descriptors(smiles)
    if desc is None:
        raise ValueError("Invalid SMILES")

    X_input_full = np.array([list(desc.values())])
    features_all = list(desc.keys())

    all_results = {}
    predicted_labels = []

    for label_name in label_cols:
        kept_indices = feature_masks[label_name]
        X_input = X_input_full[:, kept_indices]
        features = [features_all[i] for i in kept_indices]

        model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
        threshold = thresholds[label_name]
        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)

        if pred == 1:
            predicted_labels.append(label_name)
            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            shap_df = pd.DataFrame({
                "feature": features,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            all_results[label_name] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(4).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": all_results
    }

## step 3*: Explain Using All Features

In [18]:
import shap
import numpy as np
import pandas as pd
import joblib
from rdkit import Chem
from mordred import Calculator, descriptors

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"

# Load features
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()

thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

calc = Calculator(descriptors, ignore_3D=True)

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol).asdict()
    except:
        return None

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

def predict_and_explain_all_labels(smiles):
    desc = compute_descriptors(smiles)
    if desc is None:
        raise ValueError("Invalid SMILES")

    X_input = np.array([list(desc.values())])
    features = list(desc.keys())

    results = {}
    predicted_labels = []

    for label_name in label_cols:
        model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
        threshold = thresholds[label_name]

        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)

        if pred == 1:
            predicted_labels.append(label_name)

            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            shap_df = pd.DataFrame({
                "feature": features,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            results[label_name] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(4).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": results
    }


## 3** SHAP for the masked features

In [9]:
import shap
import numpy as np
import pandas as pd
import joblib
from rdkit import Chem
from mordred import Calculator, descriptors

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"

# Load full feature list
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()

thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

calc = Calculator(descriptors, ignore_3D=True)

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        desc = calc(mol).asdict()
        # Align feature order with training
        values = [desc.get(f, 0.0) for f in full_features]
        return values
    except:
        return None

def predict_and_explain_all_labels(smiles):
    desc_values = compute_descriptors(smiles)
    if desc_values is None:
        raise ValueError("Invalid SMILES")

    X_input = np.array([desc_values])
    features = full_features

    results = {}
    predicted_labels = []

    for label_name in label_cols:
        model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
        threshold = thresholds[label_name]

        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)

        if pred == 1:
            predicted_labels.append(label_name)

            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            shap_df = pd.DataFrame({
                "feature": features,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            results[label_name] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(4).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": results
    }


## step4: Summarise Explanation

In [7]:
def summarize_prediction(result):
    smiles = result["smiles"]
    predicted = result["predicted_labels"]
    if not predicted:
        print(f"The drug (SMILES: `{smiles}`) is predicted to have no significant toxicity endpoints.")
        return

    text = f"The given drug (SMILES: `{smiles}`) is predicted to be toxic in: **{', '.join(predicted)}**.\n"
    for label in predicted:
        feats = result["explanations"][label]["top_features"]
        feat_str = ", ".join([f"`{f[0]}` ({f[1]:.2f})" for f in feats])
        text += f"\n→ For **{label}**, important features include: {feat_str}"
    print(text)

In [20]:
# Example usage
result = predict_and_explain_all_labels("ClC1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)(C3=CC=CC=C3Cl)N") 
summarize_prediction(result)


The given drug (SMILES: `ClC1=CC=C(C=C1)C(C2=CC=C(C=C2)Cl)(C3=CC=CC=C3Cl)N`) is predicted to be toxic in: **NR-ER, NR-ER-LBD, SR-ARE, SR-MMP, SR-p53**.

→ For **NR-ER**, important features include: `GATS6Z` (-0.32), `GATS4s` (-0.32), `AATS8i` (0.31), `GATS8m` (-0.29)
→ For **NR-ER-LBD**, important features include: `JGI9` (0.64), `ATSC1v` (0.42), `BCUTi-1h` (-0.19), `EState_VSA9` (0.18)
→ For **SR-ARE**, important features include: `ATSC1p` (0.32), `piPC3` (0.30), `nBase` (-0.29), `SLogP` (0.28)
→ For **SR-MMP**, important features include: `SLogP` (2.01), `ETA_dEpsilon_D` (0.31), `MATS1i` (0.28), `GATS1p` (0.25)
→ For **SR-p53**, important features include: `AATS3d` (0.50), `SLogP` (0.34), `EState_VSA9` (0.33), `GATS2dv` (-0.32)


# Fine-tuned to the teeth!

Smarter Feature Selection:

* Retain per-label selection (SelectFromModel) ✅
* But use cumulative feature importance thresholding rather than median.

Better Class Imbalance Handling:
* Keep using scale_pos_weight ✅
* Additionally, apply focal loss via a custom objective for LightGBM (approximation).

Expanded Model Complexity:
* Increase n_estimators (to 1000) and tweak num_leaves/max_depth limits.

Cross-Validation-Based Thresholding:
* Calibrate optimal thresholds on validation set using F1 max or Youden’s J statistic.

Save all outputs to:
* tox21_lightgb_pipeline/models/v7

## train_lightgbm_v7.py

In [43]:
import os
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
import optuna

# === Setup paths ===
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
MODEL_PATH = "tox21_lightgb_pipeline/models/v7"
os.makedirs(MODEL_PATH, exist_ok=True)

# === Load data ===
X = np.load(os.path.join(SAVE_DIR, "X.npy"))
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    feature_names = f.read().splitlines()

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# === Saveable dictionaries ===
results = []
feature_mask_dict = {}
threshold_dict = {}

# === Loop through each label ===
for i, label_name in enumerate(label_cols):
    print(f"\n📍 Training for label: {label_name}")

    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    # === Split ===
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, stratify=y_valid, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )

    # === Feature selection ===
    selector_model = lgb.LGBMClassifier(n_estimators=300, random_state=42)
    selector_model.fit(X_train, y_train)
    importances = selector_model.feature_importances_

    sorted_idx = np.argsort(importances)[::-1]
    cum_imp = np.cumsum(importances[sorted_idx]) / np.sum(importances)
    cutoff_idx = sorted_idx[:np.searchsorted(cum_imp, 0.95)]  # Keep 95% of importance

    X_train_fs = X_train[:, cutoff_idx]
    X_val_fs = X_val[:, cutoff_idx]
    X_test_fs = X_test[:, cutoff_idx]

    feature_mask_dict[label_name] = cutoff_idx

    # === Class imbalance weight ===
    pos = np.sum(y_train == 1)
    neg = np.sum(y_train == 0)
    scale_pos_weight = neg / pos if pos > 0 else 1.0

    # === Optuna tuning ===
    def objective(trial):
        params = {
            "objective": "binary",
            "metric": "auc",
            "boosting_type": "gbdt",
            "verbosity": -1,
            "scale_pos_weight": scale_pos_weight,
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
            "num_leaves": trial.suggest_int("num_leaves", 32, 128),
            "max_depth": trial.suggest_int("max_depth", 6, 16),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "lambda_l1": trial.suggest_float("lambda_l1", 0, 1),
            "lambda_l2": trial.suggest_float("lambda_l2", 0, 1)
        }
        dtrain = lgb.Dataset(X_train_fs, label=y_train)
        dval = lgb.Dataset(X_val_fs, label=y_val)
        model = lgb.train(
            params, dtrain, valid_sets=[dval],
            callbacks=[lgb.early_stopping(stopping_rounds=30), lgb.log_evaluation(period=0)]
        )
        preds = model.predict(X_val_fs)
        return roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=40)

    # === Final model ===
    best_params = study.best_params
    best_params.update({
        "objective": "binary", "metric": "auc", "verbosity": -1,
        "boosting_type": "gbdt", "scale_pos_weight": scale_pos_weight
    })
    model = lgb.LGBMClassifier(**best_params, n_estimators=1000)
    model.fit(X_train_fs, y_train)

    # === Threshold ===
    val_probs = model.predict_proba(X_val_fs)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_val, val_probs)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
    best_thresh = thresholds[np.argmax(f1_scores)]

    # === Evaluation ===
    test_probs = model.predict_proba(X_test_fs)[:, 1]
    test_preds = (test_probs >= best_thresh).astype(int)
    auc = roc_auc_score(y_test, test_probs)
    acc = accuracy_score(y_test, test_preds)

    # === Save ===
    joblib.dump(model, os.path.join(MODEL_PATH, f"{label_name}.pkl"))
    threshold_dict[label_name] = best_thresh

    results.append({
        "label": label_name,
        "auc": auc,
        "accuracy": acc,
        "threshold": best_thresh,
        "features_used": len(cutoff_idx),
        "scale_pos_weight": scale_pos_weight
    })

# === Save metadata ===
joblib.dump(threshold_dict, os.path.join(MODEL_PATH, "thresholds.pkl"))
joblib.dump(feature_mask_dict, os.path.join(MODEL_PATH, "feature_masks.pkl"))
pd.DataFrame(results).to_csv(os.path.join(MODEL_PATH, "training_metrics.csv"), index=False)

print("✅ All models trained and saved to v7.")



📍 Training for label: NR-AR


[I 2025-08-05 14:20:48,229] A new study created in memory with name: no-name-009962cf-f31b-4c72-9280-33fb9aede734
[I 2025-08-05 14:20:48,611] Trial 0 finished with value: 0.7182640983987004 and parameters: {'learning_rate': 0.07899298862448101, 'num_leaves': 104, 'max_depth': 7, 'min_child_samples': 41, 'feature_fraction': 0.606253311898746, 'bagging_fraction': 0.7043567381064915, 'lambda_l1': 0.3542157257715218, 'lambda_l2': 0.6376884944977653}. Best is trial 0 with value: 0.7182640983987004.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[20]	valid_0's auc: 0.718264


[I 2025-08-05 14:20:48,953] Trial 1 finished with value: 0.7092364817823161 and parameters: {'learning_rate': 0.08843294172160711, 'num_leaves': 32, 'max_depth': 8, 'min_child_samples': 24, 'feature_fraction': 0.6066812014500895, 'bagging_fraction': 0.9277887004595661, 'lambda_l1': 0.9374733404035374, 'lambda_l2': 0.1656418901288551}. Best is trial 0 with value: 0.7182640983987004.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[6]	valid_0's auc: 0.709236
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.718357


...

Did not meet early stopping. Best iteration is:
[99]	valid_0's auc: 0.871541
Training until validation scores don't improve for 30 rounds


[I 2025-08-05 14:28:24,589] Trial 38 finished with value: 0.8824146981627297 and parameters: {'learning_rate': 0.07841071217826501, 'num_leaves': 79, 'max_depth': 15, 'min_child_samples': 48, 'feature_fraction': 0.7767814097682123, 'bagging_fraction': 0.7660281723923752, 'lambda_l1': 0.7657231103286853, 'lambda_l2': 0.18597929685506775}. Best is trial 27 with value: 0.9031871016122984.


Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.882415
Training until validation scores don't improve for 30 rounds


[I 2025-08-05 14:28:25,208] Trial 39 finished with value: 0.8970753655793026 and parameters: {'learning_rate': 0.09522757511077412, 'num_leaves': 111, 'max_depth': 6, 'min_child_samples': 30, 'feature_fraction': 0.7038607680248128, 'bagging_fraction': 0.8180340551383164, 'lambda_l1': 0.9450515098908956, 'lambda_l2': 0.11169215597876778}. Best is trial 27 with value: 0.9031871016122984.


Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.897075
✅ All models trained and saved to v7.


## ensemble_utils.py

In [44]:
import os
import joblib
import numpy as np
import pandas as pd

MODEL_PATH = "tox21_lightgb_pipeline/models/v7"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"

# Load metadata
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))
feature_masks = joblib.load(os.path.join(MODEL_PATH, "feature_masks.pkl"))
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

def ensemble_predict(X_input_raw):
    """
    Input: X_input_raw is a shape (n_samples, n_features) numpy array using full descriptor order.
    Returns: dict with class-wise probabilities and binary predictions.
    """
    results = {}

    for label in label_cols:
        model = joblib.load(os.path.join(MODEL_PATH, f"{label}.pkl"))
        mask = feature_masks[label]
        X_input_masked = X_input_raw[:, mask]

        prob = model.predict_proba(X_input_masked)[:, 1]
        pred = (prob >= thresholds[label]).astype(int)

        results[label] = {
            "prob": prob,
            "pred": pred
        }

    return results


## Ensemble on New Input

In [45]:
from rdkit import Chem
from mordred import Calculator, descriptors

# === Descriptor setup ===
calc = Calculator(descriptors, ignore_3D=True)
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        desc = calc(mol).asdict()
        values = [desc.get(f, 0.0) for f in full_features]
        return np.array([values])
    except:
        return None

# === Example prediction ===
smiles = "CC(=O)Oc1ccccc1C(=O)O"  # Aspirin
X_input = compute_descriptors(smiles)

if X_input is not None:
    ensemble_results = ensemble_predict(X_input)
    for label, out in ensemble_results.items():
        if out["pred"][0] == 1:
            print(f"🧪 {label} → Prob: {out['prob'][0]:.4f}")
else:
    print("Invalid SMILES")


In [47]:
# Load test data and label mask
X = np.load(os.path.join(SAVE_DIR, "X.npy"))
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))

results = []

for i, label in enumerate(label_cols):
    mask = label_mask[:, i]
    X_test = X[mask]
    y_true = Y[mask, i]

    model = joblib.load(os.path.join(MODEL_PATH, f"{label}.pkl"))
    feature_mask = feature_mask_dict[label]
    threshold = thresholds[label]

    X_test_masked = X_test[:, feature_mask]
    probs = model.predict_proba(X_test_masked)[:, 1]
    preds = (probs >= threshold).astype(int)

    auc = roc_auc_score(y_true, probs)
    acc = accuracy_score(y_true, preds)

    results.append({
        "label": label,
        "auc": auc,
        "accuracy": acc,
        "n_samples": len(y_true)
    })

# Show results
df = pd.DataFrame(results).sort_values("auc", ascending=False)
print(df)


            label       auc  accuracy  n_samples
10         SR-MMP  0.993804  0.984509       5810
2          NR-AhR  0.989432  0.983204       6549
8        SR-ATAD5  0.987609  0.992788       7072
7          SR-ARE  0.986833  0.970850       5832
11         SR-p53  0.984474  0.985090       6774
3    NR-Aromatase  0.978521  0.988662       5821
9          SR-HSE  0.975541  0.989021       6467
5       NR-ER-LBD  0.969792  0.991804       6955
1       NR-AR-LBD  0.962436  0.995709       6758
6   NR-PPAR-gamma  0.962287  0.992558       6450
4           NR-ER  0.962182  0.966252       6193
0           NR-AR  0.958156  0.993668       7265


## SHAP.py (v7)

In [31]:
import shap
import joblib
import numpy as np
import pandas as pd
from rdkit import Chem
from mordred import Calculator, descriptors
import os

# Paths
MODEL_PATH = "tox21_lightgb_pipeline/models/v7"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"

# Load full feature list
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()

# Load thresholds and feature masks
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))
feature_masks = joblib.load(os.path.join(MODEL_PATH, "feature_masks.pkl"))

# Toxicity label names
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# Descriptor calculator
calc = Calculator(descriptors, ignore_3D=True)

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        desc = calc(mol).asdict()
        values = [desc.get(f, 0.0) for f in full_features]
        return values
    except:
        return None

def predict_and_explain_all_labels(smiles):
    desc_values = compute_descriptors(smiles)
    if desc_values is None:
        raise ValueError("Invalid SMILES")

    X_full = np.array([desc_values])
    results = {}
    predicted_labels = []

    for label in label_cols:
        model = joblib.load(os.path.join(MODEL_PATH, f"{label}.pkl"))
        threshold = thresholds[label]
        kept_indices = feature_masks[label]

        X_input = X_full[:, kept_indices]
        feature_subset = [full_features[i] for i in kept_indices]

        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)
    
        if pred == 1:
            predicted_labels.append(label)

            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            shap_df = pd.DataFrame({
                "feature": feature_subset,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            results[label] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(5).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": results
    }


## Test

In [53]:
# Example SMILES input: Tamoxifen (known to be toxic in multiple pathways)
smiles = "C1CN2CCC3=CC=CC=C3C2(C4=CC=CC=C14)C=O"

X_input = compute_descriptors(smiles)  # uses full feature list

if X_input is not None:
    ensemble_results = ensemble_predict(X_input)
    predicted = []

    for label, out in ensemble_results.items():
        if out["pred"][0] == 1:
            predicted.append((label, out["prob"][0]))

    if predicted:
        print(f"\n🔍 Predicted Toxicity Labels for: {smiles}")
        for label, prob in predicted:
            print(f" - {label}: Probability = {prob:.4f}")
    else:
        print("✅ No toxicity predicted for this compound.")
else:
    print("❌ Invalid SMILES string.")


✅ No toxicity predicted for this compound.


# Fine-tune super version 

## step 1: setup & configuration 

In [54]:
import os
import numpy as np
import pandas as pd

# === Setup paths ===
BASE_DIR = "tox21_lightgb_pipeline"
DATA_DIR = os.path.join(BASE_DIR, "Data_v6", "v8")
MODEL_DIR = os.path.join(BASE_DIR, "models", "v8")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# === Load base data ===
X = np.load(os.path.join(BASE_DIR, "Data_v6", "processed", "X.npy"))
Y = np.load(os.path.join(BASE_DIR, "Data_v6", "processed", "Y.npy"))
label_mask = np.load(os.path.join(BASE_DIR, "Data_v6", "processed", "label_mask.npy"))
with open(os.path.join(BASE_DIR, "Data_v6", "processed", "feature_names.txt")) as f:
    feature_names = f.read().splitlines()

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

print("✅ Data loaded and directories ready.")


✅ Data loaded and directories ready.


## step 2: Filter Low-Variance Features (Variance Thresholding)

In [55]:
from sklearn.feature_selection import VarianceThreshold

# === Apply Variance Threshold ===
var_thresh = VarianceThreshold(threshold=0.01)  # remove features with <1% variance
X_filtered = var_thresh.fit_transform(X)
kept_mask = var_thresh.get_support()
filtered_feature_names = [f for f, keep in zip(feature_names, kept_mask) if keep]

# === Save filtered outputs ===
np.save(os.path.join(DATA_DIR, "X_filtered.npy"), X_filtered)
np.save(os.path.join(DATA_DIR, "Y.npy"), Y)
np.save(os.path.join(DATA_DIR, "label_mask.npy"), label_mask)
with open(os.path.join(DATA_DIR, "feature_names.txt"), "w") as f:
    f.write("\n".join(filtered_feature_names))

print(f"✅ Filtered features saved. {len(filtered_feature_names)} out of {len(feature_names)} features retained.")


✅ Filtered features saved. 1285 out of 1613 features retained.


## step 3: Stratified K-Fold Ensemble Training + Threshold Calibration

In [63]:
import os
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
import optuna

# === Paths ===
DATA_DIR = "tox21_lightgb_pipeline/Data_v6/v8"
MODEL_DIR = "tox21_lightgb_pipeline/models/v8"
os.makedirs(MODEL_DIR, exist_ok=True)

# === Load filtered data ===
X = np.load(os.path.join(DATA_DIR, "X_filtered.npy"))
Y = np.load(os.path.join(DATA_DIR, "Y.npy"))
label_mask = np.load(os.path.join(DATA_DIR, "label_mask.npy"))
with open(os.path.join(DATA_DIR, "feature_names.txt")) as f:
    filtered_feature_names = f.read().splitlines()

# === Setup ===
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

n_folds = 3
results = []
threshold_dict = {}

# === Loop per label ===
for i, label_name in enumerate(label_cols):
    print(f"\n📍 Label: {label_name}")
    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

    fold_aucs = []
    fold_accs = []
    fold_thresh = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_valid, y_valid)):
        print(f"\n🔁 Fold {fold+1}/{n_folds}")
        X_train, X_val = X_valid[train_idx], X_valid[val_idx]
        y_train, y_val = y_valid[train_idx], y_valid[val_idx]

        # === Class imbalance ===
        pos = np.sum(y_train == 1)
        neg = np.sum(y_train == 0)
        scale_pos_weight = neg / pos if pos > 0 else 1.0

        # === Feature selection ===
        selector = lgb.LGBMClassifier(n_estimators=300, random_state=fold)
        selector.fit(X_train, y_train)
        importances = selector.feature_importances_
        sorted_idx = np.argsort(importances)[::-1]
        cum_imp = np.cumsum(importances[sorted_idx]) / np.sum(importances)
        cutoff_idx = sorted_idx[:np.searchsorted(cum_imp, 0.95)]

        X_train_fs = X_train[:, cutoff_idx]
        X_val_fs = X_val[:, cutoff_idx]

        # Save feature mask for this fold
        np.save(os.path.join(MODEL_DIR, f"{label_name}_v8_{fold}_mask.npy"), cutoff_idx)

        # === Optuna ===
        def objective(trial):
            params = {
                "objective": "binary",
                "metric": "auc",
                "verbosity": -1,
                "boosting_type": "gbdt",
                "scale_pos_weight": scale_pos_weight,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
                "num_leaves": trial.suggest_int("num_leaves", 31, 128),
                "max_depth": trial.suggest_int("max_depth", 6, 16),
                "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
                "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
                "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
                "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
                "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0)
            }
            dtrain = lgb.Dataset(X_train_fs, label=y_train)
            dval = lgb.Dataset(X_val_fs, label=y_val)
            model = lgb.train(params, dtrain, valid_sets=[dval],
                              callbacks=[lgb.early_stopping(30), lgb.log_evaluation(0)])
            pred = model.predict(X_val_fs)
            return roc_auc_score(y_val, pred)

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=40)
        best_params = study.best_params
        best_params.update({"objective": "binary", "metric": "auc", "verbosity": -1,
                            "boosting_type": "gbdt", "scale_pos_weight": scale_pos_weight})

        # === Final model ===
        model = lgb.LGBMClassifier(**best_params, n_estimators=1000)
        model.fit(X_train_fs, y_train)
        joblib.dump(model, os.path.join(MODEL_DIR, f"{label_name}_v8_{fold}.pkl"))

        # === Threshold tuning ===
        probs = model.predict_proba(X_val_fs)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_val, probs)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)
        best_thresh = thresholds[np.argmax(f1)]

        preds = (probs >= best_thresh).astype(int)
        auc = roc_auc_score(y_val, probs)
        acc = accuracy_score(y_val, preds)

        fold_aucs.append(auc)
        fold_accs.append(acc)
        fold_thresh.append(best_thresh)

    # === Save mean threshold ===
    threshold_dict[label_name] = np.mean(fold_thresh)
    results.append({
        "label": label_name,
        "auc": np.mean(fold_aucs),
        "accuracy": np.mean(fold_accs),
        "threshold": threshold_dict[label_name],
        "n_folds": n_folds
    })

# === Save metadata ===
joblib.dump(threshold_dict, os.path.join(MODEL_DIR, "thresholds.pkl"))
pd.DataFrame(results).to_csv(os.path.join(MODEL_DIR, "training_metrics.csv"), index=False)

print("✅ v8 ensemble models with masks saved.")


📍 Label: NR-AR

🔁 Fold 1/3


[I 2025-08-05 16:44:24,573] A new study created in memory with name: no-name-b75179c5-5fec-4937-9cf5-f94c15d98704


Training until validation scores don't improve for 30 rounds


[I 2025-08-05 16:44:25,225] Trial 0 finished with value: 0.7869331859648241 and parameters: {'learning_rate': 0.06129112565869919, 'num_leaves': 80, 'max_depth': 11, 'min_child_samples': 39, 'feature_fraction': 0.6073771743444748, 'bagging_fraction': 0.7434474443875538, 'lambda_l1': 0.9237007633417299, 'lambda_l2': 0.11172455615214438}. Best is trial 0 with value: 0.7869331859648241.


Early stopping, best iteration is:
[27]	valid_0's auc: 0.786933
Training until validation scores don't improve for 30 rounds


...

Did not meet early stopping. Best iteration is:
[97]	valid_0's auc: 0.867054
Training until validation scores don't improve for 30 rounds


[I 2025-08-05 17:13:43,993] Trial 38 finished with value: 0.8598176866099156 and parameters: {'learning_rate': 0.02868561444420446, 'num_leaves': 89, 'max_depth': 12, 'min_child_samples': 48, 'feature_fraction': 0.7277725600340549, 'bagging_fraction': 0.994314564091776, 'lambda_l1': 0.7258581918504031, 'lambda_l2': 0.6639982259364412}. Best is trial 17 with value: 0.8752416272190341.


Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.859818
Training until validation scores don't improve for 30 rounds


[I 2025-08-05 17:13:45,423] Trial 39 finished with value: 0.8453016278220552 and parameters: {'learning_rate': 0.021112522237634608, 'num_leaves': 95, 'max_depth': 13, 'min_child_samples': 34, 'feature_fraction': 0.9777861219115306, 'bagging_fraction': 0.9746285373252014, 'lambda_l1': 0.42464433820032105, 'lambda_l2': 0.4556053505434872}. Best is trial 17 with value: 0.8752416272190341.


Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.845302
✅ v8 ensemble models with masks saved.


### Eval

In [64]:
import pandas as pd
import numpy as np
import os

# === Paths ===
metrics_path = "tox21_lightgb_pipeline/models/v8/training_metrics.csv"
label_mask_path = "tox21_lightgb_pipeline/Data_v6/v8/label_mask.npy"

# === Load ===
df = pd.read_csv(metrics_path)
label_mask = np.load(label_mask_path)

label_cols = df["label"].tolist()

# === Add number of samples per label ===
df["n_samples"] = [label_mask[:, i].sum() for i in range(len(label_cols))]

# === Sort and display ===
df_sorted = df.sort_values(by="accuracy", ascending=False)

print("📊 Toxicity Class Performance Summary (sorted by accuracy):")
print(df_sorted[["label", "auc", "accuracy", "n_samples"]].to_string(index=False))

# === Flag underperforming labels ===
below_90 = df[df["accuracy"] < 0.90]
if not below_90.empty:
    print("\n⚠️ Classes with accuracy < 90%:")
    print(below_90[["label", "accuracy"]].to_string(index=False))
else:
    print("\n✅ All classes are performing above 90% accuracy.")


📊 Toxicity Class Performance Summary (sorted by accuracy):
        label      auc  accuracy  n_samples
    NR-AR-LBD 0.864284  0.980912       6758
        NR-AR 0.783315  0.974397       7265
NR-PPAR-gamma 0.824241  0.964031       6450
    NR-ER-LBD 0.838593  0.961467       6955
     SR-ATAD5 0.879722  0.955741       7072
 NR-Aromatase 0.860385  0.949837       5821
       SR-HSE 0.789907  0.940160       6467
       SR-p53 0.868132  0.923384       6774
       NR-AhR 0.905663  0.915865       6549
       SR-MMP 0.920521  0.907058       5810
        NR-ER 0.714460  0.874698       6193
       SR-ARE 0.837209  0.828361       5832

⚠️ Classes with accuracy < 90%:
 label  accuracy
 NR-ER  0.874698
SR-ARE  0.828361


## step 4: SHAP, Ensemble Inference with Thresholds (for Testing SMILES)

In [67]:
import os
import joblib
import numpy as np
import shap
import pandas as pd
from rdkit import Chem
from mordred import Calculator, descriptors

# === Paths ===
MODEL_DIR = "tox21_lightgb_pipeline/models/v8"
DATA_DIR = "tox21_lightgb_pipeline/Data_v6/v8"

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# === Descriptor setup ===
calc = Calculator(descriptors, ignore_3D=True)
with open(os.path.join(DATA_DIR, "feature_names.txt")) as f:
    all_features = f.read().splitlines()

thresholds = joblib.load(os.path.join(MODEL_DIR, "thresholds.pkl"))

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        desc = calc(mol).asdict()
        values = [desc.get(f, 0.0) for f in all_features]
        return np.array(values).reshape(1, -1)
    except:
        return None

def predict_and_explain_v8(smiles):
    X = compute_descriptors(smiles)
    if X is None:
        raise ValueError("Invalid SMILES.")

    result = {"smiles": smiles, "predicted_labels": [], "explanations": {}}

    for label in label_cols:
        # Load all 3 fold models and masks
        model_path = os.path.join(MODEL_DIR, f"{label}.pkl")
        if not os.path.exists(model_path):
            continue

        fold_models = joblib.load(model_path)
        fold_preds = []
        fold_shaps = []
        feature_names_fold = []

        for i, model in enumerate(fold_models):
            mask_path = os.path.join(MODEL_DIR, f"{label}_v8_{i}_mask.npy")
            if not os.path.exists(mask_path):
                continue
            mask = np.load(mask_path)
            X_masked = X[:, mask]
            explainer = shap.Explainer(model)
            shap_vals = explainer(X_masked)
            prob = model.predict_proba(X_masked)[0, 1]

            fold_preds.append(prob)
            fold_shaps.append(shap_vals.values[0])
            feature_names_fold.append(np.array(all_features)[mask])

        if not fold_preds:
            continue

        avg_prob = np.mean(fold_preds)
        if avg_prob >= thresholds[label]:
            result["predicted_labels"].append(label)

            shap_matrix = np.stack(fold_shaps)
            avg_shap = shap_matrix.mean(axis=0)
            top_idxs = np.argsort(np.abs(avg_shap))[::-1][:5]
            top_feats = [
                (feature_names_fold[0][i], avg_shap[i]) for i in top_idxs
            ]

            result["explanations"][label] = {
                "prob": avg_prob,
                "top_features": top_feats
            }

    return result

In [69]:

# === CLI test example ===
if __name__ == "__main__":
    test_smiles = "CCOc1ccc2nc(S(N)(=O)=O)sc2c1"  # Aspirin
    output = predict_and_explain_v8(test_smiles)

    print(f"\n🔍 Predicted Toxicities for: {output['smiles']}")
    for label in output["predicted_labels"]:
        prob = output["explanations"][label]["prob"]
        print(f" - {label} → Prob: {prob:.4f}")
        for feat, val in output["explanations"][label]["top_features"]:
            print(f"    • {feat}: {val:.3f}")



🔍 Predicted Toxicities for: CCOc1ccc2nc(S(N)(=O)=O)sc2c1
